In [3]:
using CSV, DataFrames
using JuMP, Gurobi

In [4]:
retail = "C:/Users/gap_a/Documents/MIT/OPT/project/generated_data_complete.csv"
retail = DataFrame(CSV.File(retail));

In [5]:
num_rows = size(retail)[1] # 676
num_product = length(unique(retail[!,"product_id"])) # 52

39

In [6]:
demand = retail[!,"customers"]
cost = retail[!,"freight_price"]

C1 = retail[!,"comp_1"]
C2 = retail[!,"comp_2"]
C3 = retail[!,"comp_3"]

com1_cost = retail[!,"fp1"]
com2_cost = retail[!,"fp2"]
com3_cost = retail[!,"fp3"];

In [7]:
demand = reshape(demand, 12, 39)  # Reshape by column (12 columns, 39 rows)
demand = permutedims(demand)  # Reshape by row

cost = reshape(cost, 12, 39)  # Reshape by column (12 columns, 39 rows)
cost = permutedims(cost)  # Reshape by row

C1 = reshape(C1, 12, 39)  # Reshape by column (12 columns, 39 rows)
C1 = permutedims(C1)  # Reshape by row

C2 = reshape(C2, 12, 39)  # Reshape by column (12 columns, 39 rows)
C2 = permutedims(C2)  # Reshape by row

C3 = reshape(C3, 12, 39)  # Reshape by column (12 columns, 39 rows)
C3 = permutedims(C3);  # Reshape by row

In [133]:
println(demand[1,1])
println(cost[1,1])
println(C1[1,1])
println(C2[1,1])
println(C3[1,1])

88.0
13.621882369948098
88.94052289348768
143.08869437749883
44.03451426392148


# Optimization Model 1

In [54]:
p: decison variable of the pricing of each of the 52 product

Other Variables:
D: demand of our product
T_D: total demand of the product

LoadError: syntax: extra token "variable" after end of expression

In [21]:
function model1()
    model = Model(Gurobi.Optimizer);
    demand = dataframe
    C1= dataframe
    C2= dataframe
    C3= dataframe
    Cost= dataframe
    @variable(model, Z[1:num_rows,0:months] >= 0)
    
    @NLobjective(model, Max, 
        sum(sum((Z[i,t]-Cost[i,t])*demand[i,t]*(e^(-3Z[i,t]/(C1[i,t]+C2[i,t]*C3[i,t]))) for i in 1:num_rows) for t in 1:months))

    @constraint(model,[i in 1:products], Z[i,0]==0)
    for t in 1:months
        @NLconstraint(model, [i in 1:products], Z[i,t] <= 1.05*Z[i,t-1])
        @constraint(model,sum(Z[i,t] for t in 1:months)==150)
    end
    
    optimize!(model);
    objective = objective_value(model)
    return objective, value.(p)
end

model1 (generic function with 1 method)

In [8]:
num_rows = size(retail)[1] # 468
num_months = 12
num_products = length(unique(retail[!,"product_id"])) # 
print("There are ", num_product, " number of product")

There are 39 number of product

In [9]:
function gradient_f(Z)
    g_f_x =zeros(Float64, num_products, num_months)
    for i in 1:num_products
        for t in 1:num_months
            #(D[i,t]*exp(-9*Z[i,t]^2/(C1[i,t]+C2[i,t]+C3[i,t])^2)*(18*Z[i,t]*(cost[i,t]-Z[i,t])+(C1[i,t]+C2[i,t]+C3[i,t])))/(C1[i,t]+C2[i,t]+C3[i,t])^2
            g_f_x[i,t]=(D[i,t]*exp(-6*Z[i,t]/(C1[i,t]+C2[i,t]+C3[i,t]))*((C1[i,t]+C2[i,t]+C3[i,t])+6*cost[i,t]-6*Z[i,t]))/(C1[i,t]+C2[i,t]+C3[i,t])
        end
    end
    return g_f_x
end

gradient_f (generic function with 1 method)

In [ ]:
gradient_f()

In [10]:
using LinearAlgebra
D=demand
# Objective function to minimize
function f(Z)
    #sum(sum( (Z[i,t]-cost[i,t])*D[i,t]*exp(-3Z[i,t]/(C1[i,t]+C2[i,t]+C3[i,t]))^2 for t in 1:num_months) for i in 1:num_products)  
    return sum(sum( (Z[i,t]-cost[i,t])*D[i,t]*exp(-3Z[i,t]/(C1[i,t]+C2[i,t]+C3[i,t]))^2 for t in 1:num_months) for i in 1:num_products) 
end

# Gradient of the objective function


# hessian function
function hessian_t(Z)
    g2_f_x =zeros(Float64, num_products, num_months)
    for i in 1:num_products
        for t in 1:num_months
            #(18*D[i,t]*exp(-81*Z[i,t]^2/(C1[i,t]+C2[i,t]+C3[i,t])^2)*((C1[i,t]+C2[i,t]+C3[i,t])^2*(cost[i,t]-11*Z[i,t])+162*Z[i,t]^2*(cost[i,t]-Z[i,t])))/(C1[i,t]+C2[i,t]+C3[i,t])^4
            g2_f_x[i,t]=(12*D[i,t]*exp(-6*Z[i,t]/(C1[i,t]+C2[i,t]+C3[i,t]))*((C1[i,t]+C2[i,t]+C3[i,t])+3*cost[i,t]-3*Z[i,t]))/(C1[i,t]+C2[i,t]+C3[i,t])^2
        end 
    end
    return g2_f_x
end
function hessian(Z)
    n, m = size(Z)
    H = zeros(n, m, n, m)

    for i in 1:n
        for t in 1:m
            H[i, t, i, t] = (18*D[i,t]*exp(-81*Z[i,t]^2/(C1[i,t]+C2[i,t]+C3[i,t])^2)*((C1[i,t]+C2[i,t]+C3[i,t])^2*(cost[i,t]-11*Z[i,t])+162*Z[i,t]^2*(cost[i,t]-Z[i,t])))/(C1[i,t]+C2[i,t]+C3[i,t])^4 
        end
    end

    return H
end
# gradient2 function
function gradient2_f(Z)
    g2_f_x =zeros(Float64, num_products, num_months)
    for i in 1:num_products
        for t in 1:num_months
            g2_f_x[i,t]=(18*D[i,t]*exp(-81*Z[i,t]^2/(C1[i,t]+C2[i,t]+C3[i,t])^2)*((C1[i,t]+C2[i,t]+C3[i,t])^2*(cost[i,t]-11*Z[i,t])+162*Z[i,t]^2*(cost[i,t]-Z[i,t])))/(C1[i,t]+C2[i,t]+C3[i,t])^4
        end
    end
    return g2_f_x
end

# Constraint function (inequality constraint)
function g(Z)
    g_x =zeros(Float64, num_products, num_months)
    for i in 1:num_products
        for t in 1:num_months
            g_x[i,t]=Z[i,t+1]-1.05*Z[i,t]
        end
    end
    return g_x # 2x[1, 1] + x[2, 1] - 10 <= 0
end

# Gradient of the constraint function
function gradient_g(Z)
    g_g_x =zeros(Float64, num_products, num_months)
    for i in 1:num_products
        for t in 1:num_months
            g_g_x[i,t]=-1.05
        end
    end
    return g_g_x
end

function gradient2_g(Z)
    g2_g_x =zeros(Float64, num_products, num_months)
    for i in 1:num_products
        for t in 1:num_months
            g2_g_x[i,t]=1/(Z[i,t+1]-1.05*Z[i,t])^2 
        end
    end
    return g2_g_x
end


gradient2_g (generic function with 1 method)

In [14]:

# Newton's method for interior-point optimization
function newton_interior_point_method(Z0, μ₀, tolerance)
    Z = Z0
    μ = μ₀
    current_value = f(Z)
    previous_value = 10000000000
    counter=0
    Z = Float64.(Z)
    while current_value-previous_value < tolerance #all((g(Z) .+ ϵ) .< termination)#g(Z) .+ ϵ < termination
        println(counter)
        println("initial")
        println(current_value)
        hess = hessian_t(Z)#hessian(Z)####g is the error
        #print("1111")
        grad_f = gradient_f(Z)### g is the error it is because it is bacically 1/0 zerooooooo
        #print("333")
        #grad_g = gradient_g(Z)
        # Solve the Newton system incorporating the inequality constraint
        #direction = -inv(hess) * grad
        # Reshape X, grad, and hess into vectors for linear algebra operations
        
        grad_f = Float64.(grad_f)
        println("grad")
        println(grad_f[1,1])
        hess = Float64.(hess)
        println("hess")
        println(hess[1,1])
        X_vec = vec(Z)
        #print("444")
        #grad_vec = vec(grad_f)
        #print("5555")
        #hess = reshape(hess, size(Z, 1) * size(Z, 2), size(Z, 1) * size(Z, 2))
        #print("6666")
        #hess = hess + .01*I
        
        #delta_Z_vec = - (hess \ grad_vec)  # Solving linear system instead of direct inversion
        #print("7777")
        #direction = reshape(delta_Z_vec, size(Z, 1), size(Z, 2))
        direction = -hess .*grad_f
        println("z")
        println(Z[1,1])
        println("d")
        println(direction[1,1])
        Z .+=  direction
        current_value2= f(Z)
        println("before cons")
        println(current_value2)
        for i in 1:num_products
            for t in 2:num_months
                Z[i,t] = min(Z[i,t], 1.05 * Z[i, t-1]) #adddddddddddddddd this
            end
        end
        #print("44444")
        previous_value = current_value
        current_value= f(Z)
        println("after cons")
        println(current_value)
        counter +=1
        if counter >1000
            break
        end
    end
    
    return Z, current_value
end

# Initial guess and parameters (x is a 2x1 array)
Z0 =180 .+ 50 .*rand(num_products, num_months)



μ₀ = 1.0
tolerance = .01

# Run Newton's interior-point method


0.01

In [127]:
current_value=-1
for i in 1:10000
    try
        result, current_value = newton_interior_point_method(Z0, μ₀, tolerance);
        if current_value >0
            break
        end
    catch 
    end
end
print(current_value)

0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1


In [15]:
result, current_value = newton_interior_point_method(Z0, μ₀, tolerance);

0
initial
96714.26203368243
grad
-2.887696244061452
hess
-0.04386382067669632
z
212.43932328658232
d
-0.12666539021828102
before cons
96833.16683123069
after cons
101285.83550758459


In [16]:
print(current_value)

101285.83550758459

In [135]:
for i in 1:num_products
    println(result[i,:])
end


[1255.6950069313555, 569.0511052216377, 597.4653183330047, 627.3385842496549, 658.7055134621377, 691.6407891352446, 726.2228285920069, 762.5339700216073, 800.6606685226876, 355.10411161994523, 372.16391422443024, 280.3784822691647]
[963.487945081453, 205.9753247767099, 216.2740910155454, 227.08779556632268, 238.44218534463883, 250.36429461187078, 262.88250934246435, 276.02663480958756, 289.82796655006695, 304.3193648775703, 319.53533312144884, 312.060654927303]
[1787.91909724604, 1491.2706255269857, 1565.8341561080174, 1284.239772200475, 1187.1845439949761, 1078.0852984045453, 1131.9885986171832, 1188.5866057779529, 825.9244234753501, 867.2206446491176, 910.575280186809, 956.0993572751984]
[481.18176394471925, 398.5129878035616, 418.4386371937397, 439.3605690534267, 324.44089012628507, 340.66293463259933, 357.6960813642293, 375.5808854324408, 394.3599297040629, 414.07792618926607, 434.7818224987294, 456.5209136236659]
[533.7373190121009, 560.424184962706, 507.4243848396619, 532.7956040

In [211]:
new_value= f(result)
sum_product= C1[1,8] + C2[1,8] +C3[1,8]
println(sum_product)
println(cost[1,8])
println(result[1,4])
print(new_value)

335.35980390000003
14.2875
451.25878730637277
-Inf

# Baseline Model